In [1]:
# imports
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score, KFold
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

C:\Users\Daniel\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# process 2018 tax help data
ato2016_data = pd.read_excel("atoabsgovhack2018.xlsx", sheet_name="ATO Data")
abs2016_data = pd.read_excel("atoabsgovhack2018.xlsx", sheet_name="ABS Data")
txc_data = pd.read_excel("atoabsgovhack2018.xlsx", sheet_name="Tax Help Center")
txc_data.rename(columns={'Post Code': 'Postcode'}, inplace=True)
seifa_data = pd.read_excel("atoabsgovhack2018.xlsx", sheet_name="ABS SEIFA ").convert_objects(convert_numeric=True)
seifa_data.rename(columns={'Postal Area (POA) Code': 'Postcode'}, inplace=True)
seifa_data.rename(columns={'Year': 'Income year'}, inplace=True)
seifa_data["Income year"] = seifa_data["Income year"].apply(lambda x: 2015 if x == 2011 else x)
seifa_data.replace(to_replace='-', value=0, inplace=True)

# process 2017 tax help data
ato2015_data = pd.read_excel("atoabsgovhack2017.xlsx", sheet_name="Data", skiprows=0, usecols=[0,1,2,*range(3, 17)])
ato2015_data = ato2015_data.loc[ato2015_data['Income year'] == 2015]
abs2015_data = pd.read_excel("atoabsgovhack2017.xlsx", sheet_name="Data", skiprows=0, usecols=[0,1,2,*range(17, 56)])
abs2015_data = abs2015_data.loc[abs2015_data['Income year'] == 2015]

C:\Users\Daniel\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: FutureWarning: convert_objects is deprecated.  To re-infer data dtypes for object columns, use DataFrame.infer_objects()
For all other conversions use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  


In [3]:
# process 2016 ato stats
df = pd.read_excel("taxstats2016individual06taxablestatusstateterritorypostcodetaxableincome.xlsx", sheet_name="Individuals Table 6B", skiprows=2, usecols=[1, 2, 4, 37, 39, 85, 93, 107, 129])
ato2016_stats = pd.DataFrame()
ato2016_stats['average income per person'] = df[df.columns[2]]/df[df.columns[1]]
ato2016_stats['unfranked ratio'] = df[df.columns[3]]/df[df.columns[1]]
ato2016_stats['franked ratio'] = df[df.columns[4]]/df[df.columns[1]]
ato2016_stats['cgt ratio'] = df[df.columns[5]]/df[df.columns[1]]
ato2016_stats['foreign income ratio'] = df[df.columns[6]]/df[df.columns[1]]
ato2016_stats['rent ratio'] = df[df.columns[7]]/df[df.columns[1]]
ato2016_stats['business ratio'] = df[df.columns[8]]/df[df.columns[1]]

x = ato2016_stats.values
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
ato2016_stats_norm = pd.DataFrame(x_scaled, columns=ato2016_stats.columns)

ato2016_stats_norm['average'] = ato2016_stats_norm.mean(axis=1)
ato2016_stats_norm["average"] = ato2016_stats_norm["average"].apply(lambda x: 1 - x)
ato2016_stats_norm['Postcode'] = df['Postcode']
ato2016_stats_norm = ato2016_stats_norm.loc[ato2016_stats_norm['Postcode'].isin(list(range(100,9999)))]
ato2016_stats_norm['Postcode'] = ato2016_stats_norm['Postcode'].astype(np.int64)
ato2016_stats_norm['Income year'] = 2016

# process 2015 ato stats
df = pd.read_excel("taxstats2015individual06taxablestatusstateterritorypostcode.xlsx", sheet_name="Individuals Table 6B", skiprows=2, usecols=[1, 2, 4, 37, 39, 79, 87, 101, 123])
ato2015_stats = pd.DataFrame()
ato2015_stats['average income per person'] = df[df.columns[2]]/df[df.columns[1]]
ato2015_stats['unfranked ratio'] = df[df.columns[3]]/df[df.columns[1]]
ato2015_stats['franked ratio'] = df[df.columns[4]]/df[df.columns[1]]
ato2015_stats['cgt ratio'] = df[df.columns[5]]/df[df.columns[1]]
ato2015_stats['foreign income ratio'] = df[df.columns[6]]/df[df.columns[1]]
ato2015_stats['rent ratio'] = df[df.columns[7]]/df[df.columns[1]]
ato2015_stats['business ratio'] = df[df.columns[8]]/df[df.columns[1]]
ato2015_stats['Postcode'] = df['Postcode']
ato2015_stats = ato2015_stats.loc[ato2015_stats['Postcode'].isin(list(range(100,9999)))]
ato2015_stats['Postcode'] = ato2015_stats['Postcode'].astype(np.int64)
ato2015_stats['Income year'] = 2015

x = ato2015_stats.values
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
ato2015_stats_norm = pd.DataFrame(x_scaled, columns=ato2015_stats.columns)

ato2015_stats_norm['average'] = ato2015_stats_norm.mean(axis=1)
ato2015_stats_norm["average"] = ato2015_stats_norm["average"].apply(lambda x: 1 - x)
ato2015_stats_norm['Postcode'] = df['Postcode']
ato2015_stats_norm = ato2015_stats_norm.loc[ato2015_stats_norm['Postcode'].isin(list(range(100,9999)))]
ato2015_stats_norm['Postcode'] = ato2015_stats_norm['Postcode'].astype(np.int64)
ato2015_stats_norm['Income year'] = 2015

In [4]:
# join datasets
df = pd.DataFrame()
df = df.append(ato2016_data)
df = df.append(ato2015_data)
abs_data = abs2016_data.append(abs2015_data)
df = df.merge(abs_data, on=["Income year", "Postcode"], how="outer")
ato_stats = ato2016_stats_norm.append(ato2015_stats_norm)
df = df.merge(ato_stats, on=["Income year", "Postcode"], how="outer")
df = df.merge(seifa_data, on=["Income year", "Postcode"], how="outer")
df = df.merge(txc_data, on="Postcode", how="outer")
df.fillna(0, inplace=True)
# TEMPORARY: limit to 2015/2016 data
df = df[df['Income year'].isin([2016, 2015])]

df['adjusted_individuals'] = df['average'] * df['Individuals1']

x = df['adjusted_individuals'].values.reshape(-1, 1)
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
df['adjusted_individuals'] = x_scaled

df['average_bucket'] = pd.cut(df['adjusted_individuals'].values, bins=len(df['Count'].unique()), labels = list(range(0,len(df['Count'].unique()))))
count_bucket_dict = {k: v for k, v in dict(enumerate(sorted(df['Count'].unique()))).items()}
df['adjusted_count'] = df["average_bucket"].apply(lambda x: count_bucket_dict[x])

df["score"] = df['adjusted_count'] == df['Count']
df["score"] = df["score"].apply(lambda x: 1 if x else 0)

df.drop(labels=['Count', 'score', 'average_bucket','adjusted_individuals','average'], axis=1, inplace=True)
df.rename(columns={'adjusted_count': 'Count'}, inplace=True)
df['Count'] = pd.to_numeric(df['Count'])

df.head(10)

C:\Users\Daniel\Anaconda3\lib\site-packages\pandas\core\frame.py:6201: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  sort=sort)


,id_x,Income year,Postcode,Individuals1,Taxable income or loss1,Net tax,Gross interest,Net rent,Net capital gain,Total income or loss,...,foreign income ratio,franked ratio,rent ratio,unfranked ratio,Index of Relative Socio-economic Advantage and Disadvantage,Index of Relative Socio-economic Disadvantage,Index of Economic Resources,Index of Education and Occupation,Usual Resident Population,Count
2,201600800.0,2016.0,800,5464.0,3.893756e+08,101020407.0,2146701.0,-4580471.0,2352866.0,4.019841e+08,...,0.138084,0.133282,0.182721,0.063863,1096.0,1066.0,946.0,1089.0,6464.0,0.0
3,201500800.0,2015.0,800,5579.0,3.458534e+08,83997228.0,2280912.0,-4125084.0,3027094.0,3.585177e+08,...,0.200059,0.949705,0.291183,0.207608,1072.0,1060.0,952.0,1077.0,4564.0,0.0
6,201600810.0,2016.0,810,21128.0,1.367380e+09,315901076.0,9757680.0,-17414593.0,8046750.0,1.423396e+09,...,0.194314,0.230173,0.298278,0.118851,1052.0,1037.0,1014.0,1045.0,33302.0,2.0
7,201500810.0,2015.0,810,20792.0,1.326501e+09,302144669.0,11336449.0,-14253016.0,10798280.0,1.381329e+09,...,0.202761,1.000000,0.590909,0.379164,1037.0,1027.0,1008.0,1051.0,29725.0,2.0
10,201600812.0,2016.0,812,11509.0,7.282063e+08,162280073.0,4232709.0,-10799838.0,2549416.0,7.552301e+08,...,0.180394,0.232983,0.280981,0.118367,1020.0,1013.0,1013.0,997.0,18873.0,1.0
11,201500812.0,2015.0,812,11640.0,7.351788e+08,162016768.0,5336982.0,-9507909.0,4041359.0,7.632562e+08,...,0.211158,0.964341,0.324945,0.254616,1019.0,1013.0,1015.0,1007.0,19334.0,1.0
14,201600820.0,2016.0,820,13252.0,1.054697e+09,278105884.0,8262326.0,-13694450.0,11505731.0,1.096663e+09,...,0.212031,0.285497,0.358686,0.153088,1094.0,1073.0,1011.0,1085.0,19447.0,1.0
15,201500820.0,2015.0,820,13388.0,1.003861e+09,255123871.0,9119970.0,-12824734.0,16241727.0,1.046463e+09,...,0.119826,1.000000,0.352538,0.263256,1076.0,1067.0,1008.0,1081.0,19057.0,1.0
18,201600828.0,2016.0,828,733.0,5.267338e+07,13343662.0,579153.0,111775.0,423270.0,5.444836e+07,...,0.113580,0.261619,0.425228,0.137222,979.0,970.0,942.0,993.0,1201.0,0.0
19,201500828.0,2015.0,828,698.0,5.017443e+07,12410807.0,478264.0,-435598.0,1581600.0,5.248770e+07,...,0.179742,0.968138,0.318851,0.232647,972.0,970.0,958.0,973.0,1699.0,0.0


In [5]:
# specify features columns
features = df.columns[3:-3]
df[features].columns

# feature correlations
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    count_corr = df[['Count'] + list(features)].corr(method='pearson')['Count']
    display(count_corr[count_corr > 0.5])

Count                               1.000000
Individuals1                        0.884896
Taxable income or loss1             0.808361
Net tax                             0.703981
Gross interest                      0.585425
Total income or loss                0.807830
Total deductions                    0.779970
Salary or wages                     0.852976
Medicare levy                       0.793711
Medicare levy surcharge             0.716643
Total work related expenses         0.883721
HELP assessment debt2               0.739571
0-4 years                           0.883968
5-9 years                           0.877855
10-14 years                         0.866338
15-19 years                         0.867806
20-24 years                         0.821001
25-29 years                         0.830626
30-34 years                         0.864767
35-39 years                         0.888553
40-44 years                         0.882986
45-49 years                         0.873034
50-54 year

In [6]:
# create label column and train/test split
df['label'] = df['Count']
# df['label'] = (df['Count'] > 1).astype(np.int64)
df['is_train'] = np.random.uniform(0, 1, len(df)) <= .80
train, test = df[df['is_train']==True], df[df['is_train']==False]
print("Train Class Balance:", train[train['label']==0].shape[0], " / ", train[train['label']==1].shape[0])
print("Test Class Balance:", test[test['label']==0].shape[0], " / ", test[test['label']==1].shape[0])
train.head()

Train Class Balance: 3388  /  519
Test Class Balance: 898  /  142


,id_x,Income year,Postcode,Individuals1,Taxable income or loss1,Net tax,Gross interest,Net rent,Net capital gain,Total income or loss,...,rent ratio,unfranked ratio,Index of Relative Socio-economic Advantage and Disadvantage,Index of Relative Socio-economic Disadvantage,Index of Economic Resources,Index of Education and Occupation,Usual Resident Population,Count,label,is_train
2,201600800.0,2016.0,800,5464.0,3.893756e+08,101020407.0,2146701.0,-4580471.0,2352866.0,4.019841e+08,...,0.182721,0.063863,1096.0,1066.0,946.0,1089.0,6464.0,0.0,0.0,True
6,201600810.0,2016.0,810,21128.0,1.367380e+09,315901076.0,9757680.0,-17414593.0,8046750.0,1.423396e+09,...,0.298278,0.118851,1052.0,1037.0,1014.0,1045.0,33302.0,2.0,2.0,True
10,201600812.0,2016.0,812,11509.0,7.282063e+08,162280073.0,4232709.0,-10799838.0,2549416.0,7.552301e+08,...,0.280981,0.118367,1020.0,1013.0,1013.0,997.0,18873.0,1.0,1.0,True
14,201600820.0,2016.0,820,13252.0,1.054697e+09,278105884.0,8262326.0,-13694450.0,11505731.0,1.096663e+09,...,0.358686,0.153088,1094.0,1073.0,1011.0,1085.0,19447.0,1.0,1.0,True
15,201500820.0,2015.0,820,13388.0,1.003861e+09,255123871.0,9119970.0,-12824734.0,16241727.0,1.046463e+09,...,0.352538,0.263256,1076.0,1067.0,1008.0,1081.0,19057.0,1.0,1.0,True


In [7]:
# build predictive model
def baseline_model():
	model = Sequential()
	model.add(Dense(64, input_dim=64, kernel_initializer='normal', activation='relu'))
	model.add(Dense(1, kernel_initializer='normal'))
	model.compile(loss='mean_squared_error', optimizer='adam')
	return model

estimator = KerasRegressor(build_fn=baseline_model, epochs=100, batch_size=5, verbose=0)

In [8]:
# baseline model accuracy (cross validation)
kfold = KFold(n_splits=10)
scores = cross_val_score(estimator, train[features], train['label'], cv=kfold)
print("Baseline Accuracy: %0.2f (+/- %0.2f) MSE" % (scores.mean(), scores.std() * 2))

Baseline Accuracy: nan (+/- nan) MSE


In [9]:
# standardise data
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', estimator))
pipeline = Pipeline(estimators)

# standardized model accuracy (cross validation)
kfold = KFold(n_splits=10)
scores = cross_val_score(pipeline, train[features], train['label'], cv=kfold)
print("Standardized Accuracy: %0.2f (+/- %0.2f) MSE" % (scores.mean(), scores.std() * 2))

Standardized Accuracy: -1.83 (+/- 3.75) MSE
